### Ask: Is decoding accuracy for max feature higher when max feature value is higher?
Would expect that this is true if decoder is actually picking up on things
Alg:
- load decoding models (8 for 8 splits)
- load Session data using seed from decoder (same train/test splits)
- for each session_data subselect trials by value bin (use quantile values)
- evaluate test accuracy on subset of trials

### Status
Not enough balanced sessions to make this work I don't think...

In [2]:
%load_ext autoreload
%autoreload 2

import os
import numpy as np
import pandas as pd
import utils.behavioral_utils as behavioral_utils
import utils.information_utils as information_utils
import utils.visualization_utils as visualization_utils
import utils.pseudo_classifier_utils as pseudo_classifier_utils
import utils.io_utils as io_utils

import utils.glm_utils as glm_utils
from matplotlib import pyplot as plt
import utils.spike_utils as spike_utils
import utils.subspace_utils as subspace_utils
from trial_splitters.condition_trial_splitter import ConditionTrialSplitter 
from utils.session_data import SessionData

from constants.behavioral_constants import *
from constants.decoding_constants import *
from itertools import cycle

import scipy


In [6]:
PRE_INTERVAL = 500
POST_INTERVAL = 500
INTERVAL_SIZE = 50
NUM_BINS_SMOOTH = 1
EVENT = "FixationOnCross"
SESS_SPIKES_PATH = "/data/patrick_res/firing_rates/{sess_name}_firing_rates_{pre_interval}_{event}_{post_interval}_{interval_size}_bins_1_smooth.pickle"

DATA_MODE = "FiringRate"

OUTPUT_DIR = "/data/patrick_res/hyak/pseudo"


num_bins = 2

In [7]:
def filter_split(split, beh, bin_idx):
    def filt_trials(row):
        test_trials = row["TestTrials"]
        filt_test_trials = beh[
            (beh.TrialNumber.isin(test_trials)) & 
            (beh.MaxValueBin == bin_idx)
        ].TrialNumber.unique()
        if len(filt_test_trials) < 1: 
            print("nooo")
        return pd.Series({"Condition": row.Condition, "TrainTrials": [], "TestTrials": filt_test_trials})
    filt_split = split.apply(filt_trials, axis=1)
    return filt_split


def load_session_data(row, bin_idx):
    # find splits that were used for the decoding
    sess_name = row.session_name
    behavior_path = SESS_BEHAVIOR_PATH.format(sess_name=sess_name)
    beh = pd.read_csv(behavior_path)
    valid_beh = behavioral_utils.get_valid_trials(beh)
    feature_selections = behavioral_utils.get_selection_features(valid_beh)
    valid_beh = pd.merge(valid_beh, feature_selections, on="TrialNumber", how="inner")
    beh = behavioral_utils.get_feature_values_per_session(sess_name, valid_beh)
    beh = behavioral_utils.get_max_feature_value(beh)
    num_trials_per_feat = beh.groupby("MaxFeat").TrialNumber.nunique()
    num_feats = len(num_trials_per_feat)
    if num_feats < 12 or np.any(num_trials_per_feat.values < 5):
        return None

    # load firing rates
    spikes_path = SESS_SPIKES_PATH.format(
        sess_name=sess_name, 
        pre_interval=PRE_INTERVAL, 
        event=EVENT, 
        post_interval=POST_INTERVAL, 
        interval_size=INTERVAL_SIZE
    )
    frs = pd.read_pickle(spikes_path)
    frs = frs.groupby(["UnitID", "TrialNumber"]).mean().reset_index()
    # hacky, but just pretend there's one timebin. 
    frs["TimeBins"] = 0
    frs = frs.rename(columns={DATA_MODE: "Value"})

    splitter = ConditionTrialSplitter(beh, "MaxFeat", TEST_RATIO, seed=DECODER_SEED)
    session_data = SessionData(sess_name, beh, frs, splitter)
    session_data.pre_generate_splits(NUM_SPLITS)

    beh = behavioral_utils.get_max_feature_value(beh, num_bins=num_bins, quantize_bins=True)
    custom_splits  = [filter_split(next(session_data.splitter), beh, bin_idx) for _ in range(NUM_SPLITS)]
    session_data.splitter = cycle(custom_splits)

    return session_data

In [8]:
valid_sess = pd.read_pickle(SESSIONS_PATH)
models = np.load(
    os.path.join(OUTPUT_DIR, f"intertrial_agg_max_feat_models.npy"), 
    allow_pickle=True
)
accs_by_value_bin = np.empty((num_bins, NUM_SPLITS))
for bin_idx in range(num_bins):
    print(f"Processing bin {bin_idx}")
    filt_session_data = valid_sess.apply(lambda x: load_session_data(x, bin_idx), axis=1)
    filt_session_data = filt_session_data.dropna()
    print(len(filt_session_data))
    time_bins = np.zeros(1)
    accs = pseudo_classifier_utils.evaluate_model_with_data(models, filt_session_data, time_bins)
    accs_by_value_bin[bin_idx, :] = accs

Processing bin 0
nooo
nooo
nooo
nooo
nooo
nooo
nooo
nooo
nooo
nooo
nooo
nooo
nooo
nooo
nooo
nooo
nooo
nooo
21
evaluating models for bin idx 0


ValueError: a cannot be empty unless no samples are taken